In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

### Target data


In [3]:
home = pd.read_csv('./data/home.csv')
cafe = pd.read_csv('./data/cafe_2011_2020.csv')
star = pd.read_csv('./data/starbucks_2011_2020.csv')
sub = pd.read_csv('./data/sub.csv')
pop = pd.read_csv('./data/main_p.csv')

In [4]:
df = pd.DataFrame()
df

""


In [5]:
home.shape, cafe.shape, star.shape, sub.shape, pop.shape

((1190121, 20), (23500, 16), (592, 16), (2019, 6), (925, 5))

## 스타벅스 buffer 생성

In [6]:
star.head(2)

,인허가일자,상세영업상태명,폐업일자,소재지면적,소재지우편번호,소재지전체주소,도로명전체주소,도로명우편번호,사업장명,좌표정보(x),좌표정보(y),시설총규모,Start_Year,Start_Q,End_Year,End_Q
0,1998-03-04,영업,NaN,432.28,140-133,"서울특별시 용산구 청파동3가 111-35 가동 지상1,2,3층","서울특별시 용산구 청파로47길 57 (청파동3가, 가동 지상1,2,3층)",4313.0,스타벅스숙대점,197035.083960,449151.762231,432.28,1998,1,NaN,NaN
1,1999-07-15,영업,NaN,217.56,120-808,"서울특별시 서대문구 대현동 54-4 외 2필지(54-5,54-31) 신흥빌딩 1~3층","서울특별시 서대문구 이화여대길 34, 1~3층 (대현동)",3767.0,스타벅스 이대R점,195155.273886,450692.233516,217.56,1999,3,NaN,NaN


In [7]:
import pandas as pd
from pyproj import Proj, transform

# Define the transformation function
def transform_coordinates(x, y, proj_src, proj_dst):
    lon, lat = transform(proj_src, proj_dst, x, y)
    return lat, lon

# Define the source and destination projections
proj_tm = Proj('+proj=tmerc +lat_0=38 +lon_0=127.0028902777778 +k=1 +x_0=200000 +y_0=500000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
proj_wgs84 = Proj(proj='latlong', datum='WGS84')

# Apply the transformation to each row
star[['Latitude', 'Longitude']] = star.apply(
    lambda row: transform_coordinates(row['좌표정보(x)'], row['좌표정보(y)'], proj_tm, proj_wgs84), axis=1, result_type='expand'
)

# Display the transformed DataFrame
print(star.head())

/tmp/ipykernel_891491/736773167.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_src, proj_dst, x, y)


        인허가일자 상세영업상태명        폐업일자   소재지면적  소재지우편번호  \
0  1998-03-04      영업         NaN  432.28  140-133   
1  1999-07-15      영업         NaN  217.56  120-808   
2  2000-02-15      폐업  2011-01-20  528.00  135-932   
3  2000-09-30      영업         NaN  203.30  150-876   
4  2000-10-04      폐업  2011-06-22  114.38  137-713   

                                           소재지전체주소  \
0               서울특별시 용산구 청파동3가 111-35 가동 지상1,2,3층   
1  서울특별시 서대문구 대현동 54-4 외 2필지(54-5,54-31) 신흥빌딩 1~3층   
2                            서울특별시 강남구 역삼동 820-8번지   
3                             서울특별시 영등포구 여의도동 23-9   
4                   서울특별시 서초구 반포동 19-3번지 신세계백화점 2층   

                                    도로명전체주소  도로명우편번호        사업장명  \
0  서울특별시 용산구 청파로47길 57 (청파동3가, 가동 지상1,2,3층)   4313.0     스타벅스숙대점   
1           서울특별시 서대문구 이화여대길 34, 1~3층 (대현동)   3767.0   스타벅스 이대R점   
2                                       NaN      NaN     스타벅스강남점   
3              서울특별시 영등포구 국제금융로2길 28 (여의도동)   7325.0    스타벅스여의도점   
4         

In [8]:
geometry = [Point(xy) for xy in zip(star['Longitude'], star['Latitude'])]
gdf_starbucks = gpd.GeoDataFrame(star, geometry=geometry, crs="EPSG:4326")
gdf_starbucks.head(2)

,인허가일자,상세영업상태명,폐업일자,소재지면적,소재지우편번호,소재지전체주소,도로명전체주소,도로명우편번호,사업장명,좌표정보(x),좌표정보(y),시설총규모,Start_Year,Start_Q,End_Year,End_Q,Latitude,Longitude,geometry
0,1998-03-04,영업,NaN,432.28,140-133,"서울특별시 용산구 청파동3가 111-35 가동 지상1,2,3층","서울특별시 용산구 청파로47길 57 (청파동3가, 가동 지상1,2,3층)",4313.0,스타벅스숙대점,197035.083960,449151.762231,432.28,1998,1,NaN,NaN,37.544620,126.967238,POINT (126.96724 37.54462)
1,1999-07-15,영업,NaN,217.56,120-808,"서울특별시 서대문구 대현동 54-4 외 2필지(54-5,54-31) 신흥빌딩 1~3층","서울특별시 서대문구 이화여대길 34, 1~3층 (대현동)",3767.0,스타벅스 이대R점,195155.273886,450692.233516,217.56,1999,3,NaN,NaN,37.558492,126.945957,POINT (126.94596 37.55849)


In [92]:
two_buffer = gdf_starbucks.to_crs(epsg=5186).geometry.buffer(200)

In [93]:
two_coverage = two_buffer.union_all()

In [94]:
gdf_starbucks_by_year = {}

for year in gdf_starbucks['Start_Year'].unique():
    for quarter in gdf_starbucks['Start_Q'].unique():
        year_q_df = gdf_starbucks[(gdf_starbucks['Start_Year'] == year) & (gdf_starbucks['Start_Q'] == quarter)]
        
        gdf_starbucks_by_year[f"{year}.{quarter}"] = year_q_df

In [12]:
import pandas as pd
from pyproj import Proj, transform

# Define the transformation function
def transform_coordinates(x, y, proj_src, proj_dst):
    lon, lat = transform(proj_src, proj_dst, x, y)
    return lat, lon

# Define the source and destination projections
proj_tm = Proj('+proj=tmerc +lat_0=38 +lon_0=127.0028902777778 +k=1 +x_0=200000 +y_0=500000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
proj_wgs84 = Proj(proj='latlong', datum='WGS84')

# Apply the transformation to each row
cafe[['Latitude', 'Longitude']] = cafe.apply(
    lambda row: transform_coordinates(row['좌표정보(x)'], row['좌표정보(y)'], proj_tm, proj_wgs84), axis=1, result_type='expand'
)

# Display the transformed DataFrame
cafe.head()

/tmp/ipykernel_891491/4052891024.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_src, proj_dst, x, y)


,인허가일자,상세영업상태명,폐업일자,소재지면적,소재지우편번호,소재지전체주소,도로명전체주소,도로명우편번호,사업장명,좌표정보(x),좌표정보(y),시설총규모,Start_Year,Start_Q,End_Year,End_Q,Latitude,Longitude
0,1967-10-13,영업,NaN,76.02,110-836,서울특별시 종로구 종로5가 182-3 (1층),"서울특별시 종로구 종로 222 (종로5가,(1층))",3196.0,왕관 커피숍,200138.733378,452051.937703,76.02,1967,4,NaN,NaN,37.570755,127.002358
1,1970-02-02,영업,NaN,NaN,110-420,서울특별시 종로구 관수동 4-1번지 (지하1층),"서울특별시 종로구 수표로 93-1 (관수동,(지하1층))",3191.0,아사녀,198979.758848,451921.599184,112.60,1970,1,NaN,NaN,37.569580,126.989240
2,1971-02-04,폐업,2013-04-01,100.47,121-897,서울특별시 마포구 합정동 374-1번지,NaN,NaN,탑커피숍,192271.504846,449586.060150,100.47,1971,1,2013.0,2.0,37.548505,126.913331
3,1971-11-15,영업,NaN,126.00,142-803,서울특별시 강북구 미아동 190-1번지,서울특별시 강북구 도봉로 242 (미아동),1132.0,마니커피숍,202132.411671,458690.802597,126.00,1971,4,NaN,NaN,37.630568,127.024945
4,1974-03-09,폐업,2023-07-06,119.06,110-852,서울특별시 종로구 종로4가 137-3,서울특별시 종로구 종로 188 (종로4가),3195.0,원제커피전문점,199811.963354,452026.996611,119.06,1974,1,2023.0,3.0,37.570531,126.998660


In [95]:
geometry = [Point(xy) for xy in zip(cafe['Longitude'], cafe['Latitude'])]
gdf_cafe = gpd.GeoDataFrame(cafe, geometry=geometry, crs="EPSG:4326")
gdf_cafe.head(2)

,인허가일자,상세영업상태명,폐업일자,소재지면적,소재지우편번호,소재지전체주소,도로명전체주소,도로명우편번호,사업장명,좌표정보(x),좌표정보(y),시설총규모,Start_Year,Start_Q,End_Year,End_Q,Latitude,Longitude,geometry
0,1967-10-13,영업,NaN,76.02,110-836,서울특별시 종로구 종로5가 182-3 (1층),"서울특별시 종로구 종로 222 (종로5가,(1층))",3196.0,왕관 커피숍,200138.733378,452051.937703,76.02,1967,4,NaN,NaN,37.570755,127.002358,POINT (127.00236 37.57076)
1,1970-02-02,영업,NaN,NaN,110-420,서울특별시 종로구 관수동 4-1번지 (지하1층),"서울특별시 종로구 수표로 93-1 (관수동,(지하1층))",3191.0,아사녀,198979.758848,451921.599184,112.60,1970,1,NaN,NaN,37.569580,126.989240,POINT (126.98924 37.56958)


In [96]:
gdf_cafe_by_year = {}

for year in gdf_cafe['Start_Year'].unique():
    for quarter in gdf_cafe['Start_Q'].unique():
        year_q_df = gdf_cafe[(gdf_cafe['Start_Year'] == year) & (gdf_cafe['Start_Q'] == quarter)]
        
        gdf_cafe_by_year[f"{year}.{quarter}"] = year_q_df

In [100]:
gdf_starbucks['geometry'] = two_buffer

In [185]:
gdf_join_cafes = gpd.sjoin(gdf_starbucks, gdf_cafe.to_crs(epsg=5186), how='left')

TypeError: sjoin() got an unexpected keyword argument 'op'

In [165]:
gdf_join_cafes.columns

Index(['인허가일자_left', '상세영업상태명_left', '폐업일자_left', '소재지면적_left', '소재지우편번호_left',
       '소재지전체주소_left', '도로명전체주소_left', '도로명우편번호_left', '사업장명_left',
       '좌표정보(x)_left', '좌표정보(y)_left', '시설총규모_left', 'Start_Year_left',
       'Start_Q_left', 'End_Year_left', 'End_Q_left', 'Latitude_left',
       'Longitude_left', 'geometry', 'buffer', 'index_right', '인허가일자_right',
       '상세영업상태명_right', '폐업일자_right', '소재지면적_right', '소재지우편번호_right',
       '소재지전체주소_right', '도로명전체주소_right', '도로명우편번호_right', '사업장명_right',
       '좌표정보(x)_right', '좌표정보(y)_right', '시설총규모_right', 'Start_Year_right',
       'Start_Q_right', 'End_Year_right', 'End_Q_right', 'Latitude_right',
       'Longitude_right'],
      dtype='object')

In [166]:
gdf_join_cafes = gdf_join_cafes[['인허가일자_left', '상세영업상태명_left', '폐업일자_left', '사업장명_left', 
                                 'Start_Year_left', 'Start_Q_left', 'End_Year_left', 'End_Q_left',
                                 '인허가일자_right', '상세영업상태명_right', '폐업일자_right', '사업장명_right',
                                 'Start_Year_right', 'Start_Q_right', 'End_Year_right', 'End_Q_right',
                                 'geometry']]

In [167]:
gdf_join_cafes.head(2)

,인허가일자_left,상세영업상태명_left,폐업일자_left,사업장명_left,Start_Year_left,Start_Q_left,End_Year_left,End_Q_left,인허가일자_right,상세영업상태명_right,폐업일자_right,사업장명_right,Start_Year_right,Start_Q_right,End_Year_right,End_Q_right,geometry
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2018-10-16,폐업,2022-09-01,세상의모든미소,2018.0,4.0,2022.0,3.0,"POLYGON ((197704.292 549476.526, 197704.533 54..."
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2019-11-13,폐업,2022-05-03,르팽코(Le Panko BakeryCafe),2019.0,4.0,2022.0,2.0,"POLYGON ((197704.292 549476.526, 197704.533 54..."


In [168]:
num_cols = gdf_join_cafes.select_dtypes('number').columns
for i in num_cols:
    gdf_join_cafes[i] = gdf_join_cafes[i].fillna(-1).astype('int64')

In [169]:
gdf_join_cafes[num_cols]

,Start_Year_left,Start_Q_left,End_Year_left,End_Q_left,Start_Year_right,Start_Q_right,End_Year_right,End_Q_right
0,1998,1,-1,-1,2018,4,2022,3
0,1998,1,-1,-1,2019,4,2022,2
0,1998,1,-1,-1,2017,3,-1,-1
0,1998,1,-1,-1,2014,2,2021,4
0,1998,1,-1,-1,2014,2,2015,4
...,...,...,...,...,...,...,...,...
591,2020,1,-1,-1,2017,1,2022,4
591,2020,1,-1,-1,2017,1,-1,-1
591,2020,1,-1,-1,2018,1,2019,2
591,2020,1,-1,-1,2017,3,-1,-1


In [170]:
gdf_join_cafes.replace(-1, np.nan)

,인허가일자_left,상세영업상태명_left,폐업일자_left,사업장명_left,Start_Year_left,Start_Q_left,End_Year_left,End_Q_left,인허가일자_right,상세영업상태명_right,폐업일자_right,사업장명_right,Start_Year_right,Start_Q_right,End_Year_right,End_Q_right,geometry
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2018-10-16,폐업,2022-09-01,세상의모든미소,2018.0,4.0,2022.0,3.0,"POLYGON ((197704.292 549476.526, 197704.533 54..."
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2019-11-13,폐업,2022-05-03,르팽코(Le Panko BakeryCafe),2019.0,4.0,2022.0,2.0,"POLYGON ((197704.292 549476.526, 197704.533 54..."
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2017-07-31,영업,NaN,카페늘그렇듯,2017.0,3.0,NaN,NaN,"POLYGON ((197704.292 549476.526, 197704.533 54..."
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2014-05-29,폐업,2021-10-08,할리스용산더프라임점,2014.0,2.0,2021.0,4.0,"POLYGON ((197704.292 549476.526, 197704.533 54..."
0,1998-03-04,영업,NaN,스타벅스숙대점,1998,1,NaN,NaN,2014-04-28,폐업,2015-11-18,더카페,2014.0,2.0,2015.0,4.0,"POLYGON ((197704.292 549476.526, 197704.533 54..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2020-03-23,영업,NaN,스타벅스 마곡역점,2020,1,NaN,NaN,2017-02-03,폐업,2022-10-19,이디야 커피 마곡 엠밸리점,2017.0,1.0,2022.0,4.0,"POLYGON ((185241.697 551117.032, 185241.938 55..."
591,2020-03-23,영업,NaN,스타벅스 마곡역점,2020,1,NaN,NaN,2017-03-08,영업,NaN,셀렉토커피 마곡역점,2017.0,1.0,NaN,NaN,"POLYGON ((185241.697 551117.032, 185241.938 55..."
591,2020-03-23,영업,NaN,스타벅스 마곡역점,2020,1,NaN,NaN,2018-03-30,폐업,2019-05-27,놀다가개,2018.0,1.0,2019.0,2.0,"POLYGON ((185241.697 551117.032, 185241.938 55..."
591,2020-03-23,영업,NaN,스타벅스 마곡역점,2020,1,NaN,NaN,2017-09-20,영업,NaN,엘지사이언스파크 엘 카페,2017.0,3.0,NaN,NaN,"POLYGON ((185241.697 551117.032, 185241.938 55..."


In [171]:
gdf_join_cafes['사업장명_left'].value_counts()

사업장명_left
스타벅스커피 포스코사거리점    674
스타벅스 삼성역섬유센터R점    660
스타벅스커피 삼성도심공항점    652
스타벅스도심공항터미널점      647
스타벅스 국기원사거리점      619
                 ... 
스타벅스 문정법원로R         1
스타벅스 문정역점           1
스타벅스 홍대로데오점         1
스타벅스명동성당점           1
스타벅스 구파발역점          1
Name: count, Length: 589, dtype: int64

In [172]:
len(gdf_join_cafes['사업장명_left'].unique())

589

In [173]:
# # 모든 연도와 분기에 대해 카운트하기
# years = range(2011, 2021)  # 2021년까지의 연도
# quarters = range(1, 5)  # 1분기부터 4분기까지
# cafe_counts = {}

# for year in years:
#     for quarter in quarters:
#         # 스타벅스 매장이 특정 연도와 분기에 운영 중인지 확인
#         gdf_starbucks_filtered = gdf_join_cafes[
#             (gdf_join_cafes['Start_Year_left'] < year) |  # 이전 연도에 개장된 스타벅스 매장
#             ((gdf_join_cafes['Start_Year_left'] == year) & (gdf_join_cafes['Start_Q_left'] <= quarter))  # 같은 연도에 개장된 스타벅스 매장
#         ]
#         gdf_starbucks_filtered = gdf_starbucks_filtered[
#             (gdf_starbucks_filtered['End_Year_left'] > year) |  # 이후 연도에 폐업된 스타벅스 매장
#             ((gdf_starbucks_filtered['End_Year_left'] == year) & (gdf_starbucks_filtered['End_Q_left'] >= quarter)) |  # 같은 연도에 폐업된 스타벅스 매장
#             (gdf_starbucks_filtered['End_Year_left'].isna())  # 아직 폐업하지 않은 스타벅스 매장
#         ]
        
#         # 일반 카페가 특정 연도와 분기에 운영 중인지 확인
#         gdf_cafes_filtered = gdf_starbucks_filtered[
#             (gdf_starbucks_filtered['Start_Year_right'] < year) |  # 이전 연도에 개장된 일반 카페
#             ((gdf_starbucks_filtered['Start_Year_right'] == year) & (gdf_starbucks_filtered['Start_Q_right'] <= quarter))  # 같은 연도에 개장된 일반 카페
#         ]
#         gdf_cafes_filtered = gdf_cafes_filtered[
#             (gdf_cafes_filtered['End_Year_right'] > year) |  # 이후 연도에 폐업된 일반 카페
#             ((gdf_cafes_filtered['End_Year_right'] == year) & (gdf_cafes_filtered['End_Q_right'] >= quarter)) |  # 같은 연도에 폐업된 일반 카페
#             (gdf_cafes_filtered['End_Year_right'].isna())  # 아직 폐업하지 않은 일반 카페
#         ]

#         # 각 연도와 분기별로 카페의 개수를 세어 딕셔너리에 저장
#         cafe_counts[(year, quarter)] = gdf_cafes_filtered

# # 결과 확인
# for (year, quarter), df in cafe_counts.items():
#     print(f"{year}년 {quarter}분기 운영 중인 카페 수: {len(df)}")

In [174]:
# # 모든 연도와 분기에 대해 카운트하기
# years = range(2011, 2021)  # 2021년까지의 연도
# quarters = range(1, 5)  # 1분기부터 4분기까지
# cafe_counts = {}

# for year in years:
#     for quarter in quarters:
#         # 연도와 분기 조건에 따라 필터링
#         gdf_join_cafes_filtered = gdf_join_cafes[
#             (gdf_join_cafes['Start_Year_left'] < year) |  # 이전 연도에 개장된 매장
#             ((gdf_join_cafes['Start_Year_left'] == year) & (gdf_join_cafes['Start_Q_left'] <= quarter))  # 같은 연도에 개장된 매장
#         ]
#         gdf_join_cafes_filtered = gdf_join_cafes_filtered[
#             (gdf_join_cafes_filtered['End_Year_right'] > year) |  # 이후 연도에 폐업된 매장
#             ((gdf_join_cafes_filtered['End_Year_right'] == year) & (gdf_join_cafes_filtered['End_Q_right'] >= quarter)) |  # 같은 연도에 폐업된 매장
#             (gdf_join_cafes_filtered['End_Year_right'].isna())  # 아직 폐업하지 않은 매장
#         ]

#         # 각 연도와 분기별로 카페의 개수를 세어 딕셔너리에 저장
#         cafe_counts[(year, quarter)] = gdf_join_cafes_filtered['사업장명_left'].value_counts()

# # 결과 확인
# for (year, quarter), counts in cafe_counts.items():
#     print(f"{year}년 {quarter}분기 사업장명별 카페 수:\n{counts}\n")

In [175]:
# import pandas as pd

# # 모든 연도와 분기에 대해 value_counts() 결과 저장
# years = range(2011, 2021)  # 2021년까지의 연도
# quarters = range(1, 5)  # 1분기부터 4분기까지
# cafe_counts = {}

# # 결과를 저장할 빈 데이터프레임 생성
# result_df = pd.DataFrame(columns=['Year', 'Quarter', '사업장명', 'Count'])

# for year in years:
#     for quarter in quarters:
#         # 연도와 분기 조건에 따라 필터링
#         gdf_join_cafes_filtered = gdf_join_cafes[
#             (gdf_join_cafes['Start_Year_left'] < year) |  # 이전 연도에 개장된 매장
#             ((gdf_join_cafes['Start_Year_left'] == year) & (gdf_join_cafes['Start_Q_left'] <= quarter))  # 같은 연도에 개장된 매장
#         ]
#         gdf_join_cafes_filtered = gdf_join_cafes_filtered[
#             (gdf_join_cafes_filtered['End_Year_right'] > year) |  # 이후 연도에 폐업된 매장
#             ((gdf_join_cafes_filtered['End_Year_right'] == year) & (gdf_join_cafes_filtered['End_Q_right'] >= quarter)) |  # 같은 연도에 폐업된 매장
#             (gdf_join_cafes_filtered['End_Year_right'].isna())  # 아직 폐업하지 않은 매장
#         ]

#         # 각 연도와 분기별로 카페의 개수를 세어 딕셔너리에 저장
#         value_counts = gdf_join_cafes_filtered['사업장명_left'].value_counts()
        
#         # value_counts를 DataFrame으로 변환하고, Year, Quarter 열 추가
#         counts_df = value_counts.reset_index()
#         counts_df.columns = ['사업장명', 'Count']
#         counts_df['Year'] = year
#         counts_df['Quarter'] = quarter

#         # 결과 데이터프레임에 추가
#         result_df = pd.concat([result_df, counts_df], ignore_index=True)

# # 결과 확인
# result_df

In [176]:
import pandas as pd

# 모든 연도와 분기에 대해 value_counts() 결과 저장
years = range(2011, 2021)  # 2011년부터 2020년까지의 연도
quarters = range(1, 5)  # 1분기부터 4분기까지
cafe_counts = {}

# 결과를 저장할 빈 데이터프레임 생성
result_df = pd.DataFrame(columns=['Year', 'Quarter', '사업장명', 'Count', 'geometry'])

for year in years:
    for quarter in quarters:
        # 연도와 분기 조건에 따라 필터링
        gdf_join_cafes_filtered = gdf_join_cafes[
            (gdf_join_cafes['Start_Year_left'] < year) |  # 이전 연도에 개장된 매장
            ((gdf_join_cafes['Start_Year_left'] == year) & (gdf_join_cafes['Start_Q_left'] <= quarter))  # 같은 연도에 개장된 매장
        ]
        gdf_join_cafes_filtered = gdf_join_cafes_filtered[
            (gdf_join_cafes_filtered['End_Year_right'] > year) |  # 이후 연도에 폐업된 매장
            ((gdf_join_cafes_filtered['End_Year_right'] == year) & (gdf_join_cafes_filtered['End_Q_right'] >= quarter)) |  # 같은 연도에 폐업된 매장
            (gdf_join_cafes_filtered['End_Year_right'].isna())  # 아직 폐업하지 않은 매장
        ]

        # 각 연도와 분기별로 카페의 개수를 세어 딕셔너리에 저장
        value_counts = gdf_join_cafes_filtered['사업장명_left'].value_counts()
        
        # value_counts를 DataFrame으로 변환하고, Year, Quarter 열 추가
        counts_df = value_counts.reset_index()
        counts_df.columns = ['사업장명', 'Count']
        counts_df['Year'] = year
        counts_df['Quarter'] = quarter

        # 사업장명별 대표적인 geometry 정보 추가
        # '사업장명_left' 별로 첫 번째 'geometry'를 가져옴
        geometry_info = gdf_join_cafes_filtered.groupby('사업장명_left')['geometry'].first().reset_index()
        
        # counts_df와 geometry_info를 병합
        counts_df = counts_df.merge(geometry_info, left_on='사업장명', right_on='사업장명_left', how='left')
        counts_df.drop(columns=['사업장명_left'], inplace=True)

        # 결과 데이터프레임에 추가
        result_df = pd.concat([result_df, counts_df], ignore_index=True)

# 결과 확인
result_df

/tmp/ipykernel_891491/170325528.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, counts_df], ignore_index=True)


,Year,Quarter,사업장명,Count,geometry
0,2011,1,스타벅스도심공항터미널점,589,"POLYGON ((205840.165 545589.863, 205840.406 54..."
1,2011,1,스타벅스 코엑스2점,561,"POLYGON ((205799.792 545915.02, 205800.033 545..."
2,2011,1,스타벅스삼성역점,556,"POLYGON ((206187.097 545698.877, 206187.337 54..."
3,2011,1,스타벅스 삼성현대힐점,539,"POLYGON ((205260.605 546025.238, 205260.846 54..."
4,2011,1,스타벅스 외환은행본점,343,"POLYGON ((199280.398 551798.019, 199280.639 55..."
...,...,...,...,...,...
16060,2020,4,스타벅스서울타워점,2,"POLYGON ((199663.186 549955.745, 199663.427 54..."
16061,2020,4,스타벅스올림픽프라자점,2,"POLYGON ((212288.511 546288.41, 212288.751 546..."
16062,2020,4,스타벅스 올림픽훼미리점,2,"POLYGON ((210895.895 543592.253, 210896.136 54..."
16063,2020,4,스타벅스 동부이촌동점,2,"POLYGON ((197863.782 546864.431, 197864.023 54..."


In [177]:
len(result_df['사업장명'].unique())

579

In [180]:
len(gdf_join_cafes['사업장명_left'].unique())

589

In [194]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 빈 DataFrame에 기본 값 설정
result_df = pd.DataFrame(columns=['Year', 'Quarter', '사업장명', 'Count', 'geometry'])
result_df['geometry'] = pd.Series(dtype='geometry')  # 'geometry' 열 초기화

for year in years:
    for quarter in quarters:
        # 연도와 분기 조건에 따라 필터링
        gdf_join_cafes_filtered = gdf_join_cafes[
            (gdf_join_cafes['Start_Year_left'] < year) |  # 이전 연도에 개장된 매장
            ((gdf_join_cafes['Start_Year_left'] == year) & (gdf_join_cafes['Start_Q_left'] <= quarter))  # 같은 연도에 개장된 매장
        ]
        gdf_join_cafes_filtered = gdf_join_cafes_filtered[
            (gdf_join_cafes_filtered['End_Year_right'] > year) |  # 이후 연도에 폐업된 매장
            ((gdf_join_cafes_filtered['End_Year_right'] == year) & (gdf_join_cafes_filtered['End_Q_right'] >= quarter)) |  # 같은 연도에 폐업된 매장
            (gdf_join_cafes_filtered['End_Year_right'].isna())  # 아직 폐업하지 않은 매장
        ]

        # 각 연도와 분기별로 카페의 개수를 세어 딕셔너리에 저장
        value_counts = gdf_join_cafes_filtered['사업장명_left'].value_counts()
        
        # value_counts를 DataFrame으로 변환하고, Year, Quarter 열 추가
        counts_df = value_counts.reset_index()
        counts_df.columns = ['사업장명', 'Count']
        counts_df['Year'] = year
        counts_df['Quarter'] = quarter

        if not counts_df.empty:
            # 사업장명별 대표적인 geometry 정보 추가
            geometry_info = gdf_join_cafes_filtered.groupby('사업장명_left')['geometry'].first().reset_index()
            
            # counts_df와 geometry_info를 병합
            counts_df = counts_df.merge(geometry_info, left_on='사업장명', right_on='사업장명_left', how='left')
            counts_df.drop(columns=['사업장명_left'], inplace=True)

            # 결과 데이터프레임에 추가
            result_df = pd.concat([result_df, counts_df], ignore_index=True)

# 결과 확인
result_df

,Year,Quarter,사업장명,Count,geometry
0,2011,1,스타벅스도심공항터미널점,589,"POLYGON ((205840.165 545589.863, 205840.406 54..."
1,2011,1,스타벅스 코엑스2점,561,"POLYGON ((205799.792 545915.02, 205800.033 545..."
2,2011,1,스타벅스삼성역점,556,"POLYGON ((206187.097 545698.877, 206187.337 54..."
3,2011,1,스타벅스 삼성현대힐점,539,"POLYGON ((205260.605 546025.238, 205260.846 54..."
4,2011,1,스타벅스 외환은행본점,343,"POLYGON ((199280.398 551798.019, 199280.639 55..."
...,...,...,...,...,...
16060,2020,4,스타벅스서울타워점,2,"POLYGON ((199663.186 549955.745, 199663.427 54..."
16061,2020,4,스타벅스올림픽프라자점,2,"POLYGON ((212288.511 546288.41, 212288.751 546..."
16062,2020,4,스타벅스 올림픽훼미리점,2,"POLYGON ((210895.895 543592.253, 210896.136 54..."
16063,2020,4,스타벅스 동부이촌동점,2,"POLYGON ((197863.782 546864.431, 197864.023 54..."


In [195]:
result_df.to_csv('./data/cafe_in_starbucks.csv', index=False)

In [197]:
home

,pnu,x,y,count,yyyymmdd,area,price,withoutRent,deposit,rent,builtyear,시군구,번지,도로조건,전월세구분,도로명,군,구,year,quarter
0,1111017000,955631.4,1954531.4,1372,2011-01-01,74.38,3000,1,3000,0,1967.0,서울특별시 종로구 명륜1가,9*,12m미만,전세,혜화로3길,종로구,명륜1가,2011,1
1,1165010800,957313.0,1943379.7,12504,2011-01-01,10.00,4971,0,400,32,1989.0,서울특별시 서초구 서초동,1***,-,월세,효령로47길,서초구,서초동,2011,1
2,1130510100,959067.0,1957081.1,31032,2011-01-01,50.76,5456,0,500,38,1992.0,서울특별시 강북구 미아동,5*,-,월세,오패산로,강북구,미아동,2011,1
3,1120010700,959116.1,1950895.6,7774,2011-01-01,25.00,11000,1,11000,0,2003.0,서울특별시 성동구 행당동,*,-,전세,마조로7길,성동구,행당동,2011,1
4,1120010700,958698.6,1951037.7,7774,2011-01-01,56.10,9500,1,9500,0,NaN,서울특별시 성동구 행당동,2**,-,전세,무학봉18길,성동구,행당동,2011,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190116,1138010300,950446.4,1958347.1,14436,2020-03-31,20.27,6196,0,500,30,2012.0,서울특별시 은평구 불광동,1**,-,월세,NaN,은평구,불광동,2020,1
1190117,1138010300,950514.4,1958196.1,14436,2020-03-31,32.30,10493,0,1000,50,1967.0,서울특별시 은평구 불광동,2**,8m미만,월세,통일로72길,은평구,불광동,2020,1
1190118,1144012000,948592.6,1950744.0,6930,2020-03-31,135.00,10000,1,10000,0,1997.0,서울특별시 마포구 서교동,3**,8m미만,전세,월드컵북로1길,마포구,서교동,2020,1
1190119,1117010900,952933.5,1950049.7,1928,2020-03-31,33.00,4200,1,4200,0,1991.0,서울특별시 용산구 청파동1가,8*,-,전세,NaN,용산구,청파동1가,2020,1


In [206]:
region_map = {'종로구': '도심권',
              '중구': '도심권', 
              '용산구': '도심권',
              '강북구': '동북권',
              '도봉구': '동북권',
              '노원구': '동북권', 
              '성북구': '동북권', 
              '중랑구': '동북권', 
              '동대문구': '동북권', 
              '성동구': '동북권',
              '광진구': '동북권',
              '서초구': '동남권', 
              '강남구': '동남권', 
              '송파구': '동남권', 
              '강동구': '동남권',
              '은평구': '서북권', 
              '서대문구': '서북권', 
              '마포구': '서북권',
              '강서구': '서남권', 
              '양천구': '서남권', 
              '영등포구': '서남권', 
              '구로구': '서남권', 
              '금천구': '서남권', 
              '동작구': '서남권', 
              '관악구': '서남권'}


home['범주'] = home['군'].map(region_map)

In [208]:
home['범주']

0          도심권
1          동남권
2          동북권
3          동북권
4          동북권
          ... 
1190116    서북권
1190117    서북권
1190118    서북권
1190119    도심권
1190120    동북권
Name: 범주, Length: 1190121, dtype: object

In [215]:
home['yyyymmdd'] = pd.to_datetime(home['yyyymmdd'])
home['month'] = home['yyyymmdd'].dt.month

In [328]:
region_df = pd.read_csv('./data/전월세전환율_202003.tsv', sep='\t', encoding='cp949')
region_df

,type,name,code,yyyymm,value
0,APT,서울특별시,11,201101,7.40
1,APT,종로구,11110,201101,7.60
2,APT,중구,11140,201101,7.40
3,APT,용산구,11170,201101,7.20
4,APT,성동구,11200,201101,7.20
...,...,...,...,...,...
26635,OFCT,인천광역시,28,201911,7.71
26636,OFCT,인천광역시,28,201912,7.66
26637,OFCT,인천광역시,28,202001,7.58
26638,OFCT,인천광역시,28,202002,7.57


In [334]:
region_df[region_df['name'] == '성북구']['type'].unique()

array(['APT'], dtype=object)

In [335]:
region_df = region_df[region_df['type'] == 'DD']

In [336]:
region_df = region_df[region_df['name'].isin(region_map.keys())]

In [337]:
region_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2442 entries, 196 to 26290
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   type    2442 non-null   object 
 1   name    2442 non-null   object 
 2   code    2442 non-null   int64  
 3   yyyymm  2442 non-null   int64  
 4   value   2442 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 114.5+ KB


In [338]:
home['Month'] = home['month'].apply(lambda x: f"{x:02d}")

# 연도와 월을 결합하여 문자열 생성
home['yyyymm'] = home['year'].astype('str') + home['Month']

In [288]:
home.drop('범주', axis=1, inplace=True)

In [304]:
home['transit'] = 0

In [316]:
region_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2442 entries, 196 to 26290
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   type    2442 non-null   object 
 1   name    2442 non-null   object 
 2   code    2442 non-null   int64  
 3   yyyymm  2442 non-null   int64  
 4   value   2442 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 114.5+ KB


In [317]:
home.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190121 entries, 0 to 1190120
Data columns (total 24 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   pnu          1190121 non-null  int64         
 1   x            1190121 non-null  float64       
 2   y            1190121 non-null  float64       
 3   count        1190121 non-null  int64         
 4   yyyymmdd     1190121 non-null  datetime64[ns]
 5   area         1190121 non-null  float64       
 6   price        1190121 non-null  int64         
 7   withoutRent  1190121 non-null  int64         
 8   deposit      1190121 non-null  int64         
 9   rent         1190121 non-null  int64         
 10  builtyear    1111759 non-null  float64       
 11  시군구          1190121 non-null  object        
 12  번지           1190023 non-null  object        
 13  도로조건         1190052 non-null  object        
 14  전월세구분        1190121 non-null  object        
 15  도로명          11

In [327]:
region_df[region_df['name'] == '도봉구']

,type,name,code,yyyymm,value


In [364]:
from tqdm.auto import tqdm

In [418]:
# 필요한 데이터 미리 준비
unique_yyyymm = home['yyyymm'].unique()
home['yyyymm'] = home['yyyymm'].astype(int)
region_df['yyyymm'] = region_df['yyyymm'].astype(int)

# 각 지역에 대해 한 번에 계산
for i in region_map.keys():
    # 해당 지역의 데이터만 필터링
    home_region = home[home['군'] == i]
    region_value_df = region_df[region_df['name'] == i]

    # yyyymm에 대한 join 연산
    merged = home_region.merge(region_value_df[['yyyymm', 'value']], on='yyyymm', how='left')

    # # NaN 값을 0으로 채우기
    merged['value'] = merged['value'].fillna(0)

    # # # transit 계산
    merged['transit'] = np.where(
        merged['withoutRent'] == 1,
        (merged['deposit'] * merged['value'] / 12),
        (merged['deposit'] * merged['value'] / 12) + merged['rent']
    )

    merged.index = home_region.index
    # # # 결과를 원본 데이터프레임에 업데이트
    home.loc[home['군'] == i, 'transit'] = merged['transit']

    # print(f"Processed {i}")


# float 타입으로 변환
home['transit'] = home['transit'].astype('float')

In [420]:
home[home['transit'] == 0]

,pnu,x,y,count,yyyymmdd,area,price,withoutRent,deposit,rent,...,전월세구분,도로명,군,구,year,quarter,month,transit,Month,yyyymm
20,1135010500,962113.5,1963578.2,17928,2011-01-01,47.14,4000,1,4000,0,...,전세,덕릉로126라길,노원구,상계동,2011,1,1,0.0,01,201101
73,1132010600,958763.3,1962644.3,10018,2011-01-01,157.90,5000,1,5000,0,...,전세,도봉로153길,도봉구,방학동,2011,1,1,0.0,01,201101
81,1129013300,955860.1,1957455.9,9510,2011-01-01,66.00,6000,1,6000,0,...,전세,정릉로21길,성북구,정릉동,2011,1,1,0.0,01,201101
120,1135010200,961082.3,1958715.4,4699,2011-01-02,13.00,5000,1,5000,0,...,전세,광운로19가길,노원구,월계동,2011,1,1,0.0,01,201101
121,1135010200,960952.4,1958765.0,4699,2011-01-02,51.41,4000,1,4000,0,...,전세,월계로42길,노원구,월계동,2011,1,1,0.0,01,201101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190043,1129013800,960036.6,1957139.3,14129,2020-03-31,87.83,18000,1,18000,0,...,전세,장월로14길,성북구,장위동,2020,1,3,0.0,03,202003
1190048,1129012000,957346.1,1955330.3,882,2020-03-31,56.61,21000,1,21000,0,...,전세,동소문로23길,성북구,동선동5가,2020,1,3,0.0,03,202003
1190061,1132010500,958117.0,1961782.4,9996,2020-03-31,14.98,4000,1,4000,0,...,전세,노해로41나길,도봉구,쌍문동,2020,1,3,0.0,03,202003
1190068,1129013300,955661.1,1957475.0,9510,2020-03-31,55.39,10000,1,10000,0,...,전세,정릉로21길,성북구,정릉동,2020,1,3,0.0,03,202003


In [421]:
home.to_csv('./data/transit_home.csv', index=False)